<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Copy_of_Clean_Fasttext_10_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install unidecode
!pip install word2number
!pip install contractions

     |████████████████████████████████| 235 kB 4.3 MB/s 
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=0ae776de1e602cba2a6ff74886f6179b2b58ffb05908df75ba84a6096de0b671
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 321 kB 4.3 MB/s 
     |████████████████████████████████| 284 kB 33.1 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=7c8f1929b73e7b6fe90a41d8e44a139b8cc0ff8d6f26d733cf37fc67733ee6c0
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [10]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [5]:
dforiginal = df_fake.head(5).append(df_real.head(5))

In [13]:
df= df_fake.head(10).append(df_real.head(10))
#df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [14]:
#print(df.columns)
##df.count()
df

,id,news_url,title,tweet_ids,labelML
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,0
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,0
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,0
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,0
5,gossipcop-5189580095,www.townandcountrymag.com/society/tradition/a1...,Here's What Really Happened When JFK Jr. Met P...,890253005299351552\t890401381814870016\t890491...,0
6,gossipcop-9588339534,www.foxnews.com/entertainment/2016/12/16/bigge...,Biggest celebrity scandals of 2016,683226380742557696\t748604615503929345\t748604...,0
7,gossipcop-8753274298,www.eonline.com/news/958257/caitlyn-jenner-add...,Caitlyn Jenner Addresses Rumored Romance With ...,1026891446081728512\t1026891745219543043\t1026...,0
8,gossipcop-8105333868,www.inquisitr.com/3871816/taylor-swift-reporte...,Taylor Swift Reportedly Reacts To Tom Hiddlest...,818928533569437697\t819100640878202880\t819174...,0
9,gossipcop-2803748870,www.huffingtonpost.com/entry/kate-mckinnon-the...,"For The Love Of God, Why Can't Anyone Write Ka...",816030248190046212\t816030859484626947\t816049...,0


In [15]:
print(df)

                      id  ... labelML
0   gossipcop-2493749932  ...       0
1   gossipcop-4580247171  ...       0
2    gossipcop-941805037  ...       0
3   gossipcop-2547891536  ...       0
4   gossipcop-5476631226  ...       0
5   gossipcop-5189580095  ...       0
6   gossipcop-9588339534  ...       0
7   gossipcop-8753274298  ...       0
8   gossipcop-8105333868  ...       0
9   gossipcop-2803748870  ...       0
10      gossipcop-882573  ...       1
11      gossipcop-875924  ...       1
12      gossipcop-894416  ...       1
13      gossipcop-857248  ...       1
14      gossipcop-884684  ...       1
15      gossipcop-899309  ...       1
16      gossipcop-848092  ...       1
17      gossipcop-897293  ...       1
18      gossipcop-926957  ...       1
19      gossipcop-930832  ...       1

[20 rows x 5 columns]


In [16]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      20
labelML    20
dtype: int64

In [14]:
# FOR URL Preprocessing now. 
dfurl = df.drop(['tweet_ids','id'], axis=1)
dfurl.head()

,news_url,title,labelML
0,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,0
1,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,0
2,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,0
3,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,0
4,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,0


In [25]:
def remove_URL(sample):
    print(type(sample))
    if type(sample) == "str":
      return re.sub(r"http\S+", "", sample)

In [ ]:
#dfurl['news_url'] = dfurl['news_url'].apply(lambda x: ' '.join(remove_URL(x)))

In [17]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [18]:
print(df1)
#df1.count()

                                                title  labelML
0   did miley cyrus and liam hemsworth secretly ge...        0
1   paris jackson cara delevingne enjoy night out ...        0
2   celebrities join tax march in protest of donal...        0
3   cindy crawford daughter kaia gerber wears wig ...        0
4              full list of oscar nominations variety        0
5   here what really happened when jfk jr met prin...        0
6                       biggest celebrity scandals of        0
7   caitlyn jenner addresses rumored romance with ...        0
8   taylor swift reportedly reacts to tom hiddlest...        0
9   for the love of god why can anyone write kate ...        0
10  teen mom star jenelle evans wedding dress is a...        1
11  kylie jenner refusing to discuss tyga on life ...        1
12                                      quinn perkins        1
13  tried kim kardashian butt workout am forever c...        1
14  celine dion donates concert proceeds to vegas ...  

In [19]:
print(df1)

                                                title  labelML
0   did miley cyrus and liam hemsworth secretly ge...        0
1   paris jackson cara delevingne enjoy night out ...        0
2   celebrities join tax march in protest of donal...        0
3   cindy crawford daughter kaia gerber wears wig ...        0
4              full list of oscar nominations variety        0
5   here what really happened when jfk jr met prin...        0
6                       biggest celebrity scandals of        0
7   caitlyn jenner addresses rumored romance with ...        0
8   taylor swift reportedly reacts to tom hiddlest...        0
9   for the love of god why can anyone write kate ...        0
10  teen mom star jenelle evans wedding dress is a...        1
11  kylie jenner refusing to discuss tyga on life ...        1
12                                      quinn perkins        1
13  tried kim kardashian butt workout am forever c...        1
14  celine dion donates concert proceeds to vegas ...  

In [20]:
df1.count()

title      20
labelML    20
dtype: int64

In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2022-02-02 07:28:09--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  25.3MB/s    in 2m 51s  

2022-02-02 07:31:01 (25.1 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



trying to see similarity of vectors created.

In [3]:
!pip install fasttext
import fasttext as ft

!pip3 install pyfasttext
from pyfasttext import FastText

# Loading model for English
modelog = ft.load_model('cc.en.300.bin') 
 
# Getting word vectors for 'one' and 'two'.
one = modelog.get_word_vector('one')
two = modelog.get_word_vector('two')

# Getting the sentence vector for the sentence "one two" in English.
one_two = modelog.get_sentence_vector('one two')
one_two_avg = (one + two) / 2

In [4]:
c = modelog.get_sentence_vector('paris hiltion liam hemswoth go to hospital')
d = modelog.get_sentence_vector('miley cyrus liam hemswoth go to hospital')

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
cosine_similarity = 1 - spatial.distance.cosine(c, d)
cosine_similarity

0.8526310920715332

In [6]:
modelog.get_nearest_neighbors('hello')

[(0.7143728733062744, 'hellow'),
 (0.7095366716384888, 'hello.'),
 (0.703833818435669, 'hi'),
 (0.6944323182106018, 'hullo'),
 (0.6912142038345337, 'hello-'),
 (0.663975715637207, 'Hello'),
 (0.6563249230384827, 'hello.I'),
 (0.6529381275177002, 'howdy'),
 (0.6283847689628601, 'hellooo'),
 (0.6235803365707397, 'hellooooo')]

In [24]:
embedding=[]
for line in df1['title']:
    pred_label=modelog.get_sentence_vector(line)
    embedding.append(pred_label)

# you add the list to the dataframe, then save the datframe to new csv
df1['prediction']=embedding
df1.to_csv('csv_word_embed_gossipcop.csv',index=False)

In [25]:
df1

,title,labelML,prediction
0,did miley cyrus and liam hemsworth secretly ge...,0,"[0.00040790025, -0.0069974544, 0.043367695, 0...."
1,paris jackson cara delevingne enjoy night out ...,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060..."
2,celebrities join tax march in protest of donal...,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0..."
3,cindy crawford daughter kaia gerber wears wig ...,0,"[0.021224087, 0.003118416, -0.035745192, 0.044..."
4,full list of oscar nominations variety,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112..."
5,here what really happened when jfk jr met prin...,0,"[0.0038485283, 0.026331771, -0.017308457, 0.04..."
6,biggest celebrity scandals of,0,"[-0.0499622, -0.0005422188, -0.016689636, 0.05..."
7,caitlyn jenner addresses rumored romance with ...,0,"[-0.012884307, 0.0034993952, -0.033565678, 0.0..."
8,taylor swift reportedly reacts to tom hiddlest...,0,"[-0.011871912, 0.010264976, 0.025739789, 0.057..."
9,for the love of god why can anyone write kate ...,0,"[-0.006167959, 0.006165242, 0.017993517, 0.051..."


In [26]:
type(df1)

pandas.core.frame.DataFrame

In [27]:
df3 = df1.drop(['title'], axis=1)
df3

,labelML,prediction
0,0,"[0.00040790025, -0.0069974544, 0.043367695, 0...."
1,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060..."
2,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0..."
3,0,"[0.021224087, 0.003118416, -0.035745192, 0.044..."
4,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112..."
5,0,"[0.0038485283, 0.026331771, -0.017308457, 0.04..."
6,0,"[-0.0499622, -0.0005422188, -0.016689636, 0.05..."
7,0,"[-0.012884307, 0.0034993952, -0.033565678, 0.0..."
8,0,"[-0.011871912, 0.010264976, 0.025739789, 0.057..."
9,0,"[-0.006167959, 0.006165242, 0.017993517, 0.051..."


In [28]:
for index, arr in df3['prediction'].iteritems():
  for i in range(len(arr)):
    df3.loc[index, str(i)] = arr[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [75]:
#df3.to_pickle('ourpickle.pkl', compression=None)

In [29]:
df3

,labelML,prediction,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0,"[0.00040790025, -0.0069974544, 0.043367695, 0....",0.000408,-0.006997,0.043368,0.059105,0.005750,-0.048923,-0.023493,-0.012002,0.008741,0.037747,-0.016657,0.021009,-0.000813,-0.005880,0.021754,0.012577,-0.026488,0.006688,-0.065610,0.031214,0.000651,0.027043,0.026573,-0.008683,0.011755,-0.033109,-0.012075,0.001186,-0.013979,0.112681,0.002063,-0.028648,-0.023800,0.026196,-0.012809,0.025506,0.010329,0.047436,...,-0.010071,0.012518,0.041530,0.023292,-0.059176,0.013486,0.009417,0.022773,-0.007663,-0.022772,0.013969,0.001931,-0.034058,0.000787,0.023271,0.028486,0.013586,0.034805,-0.002466,0.012882,-0.038919,-0.005540,-0.085419,-0.050894,0.005842,0.044290,0.008440,-0.039907,-0.004749,0.006133,0.010734,0.009146,-0.105409,0.021976,-0.012046,0.027996,0.003418,0.054863,-0.031841,-0.042299
1,0,"[0.0054201577, -0.003211912, 0.01313437, 0.060...",0.005420,-0.003212,0.013134,0.060644,-0.007347,-0.006692,0.008933,-0.000213,0.008129,0.027304,-0.039039,-0.017705,-0.002902,-0.014937,0.031546,-0.008590,-0.023857,-0.014054,-0.032486,-0.016127,-0.006829,0.010045,0.028259,0.022507,-0.009351,-0.032367,-0.010367,-0.022886,-0.008200,0.070938,0.022677,-0.004803,-0.024451,-0.023938,-0.018909,0.003604,-0.009259,0.074453,...,0.007414,0.007728,0.017345,0.005856,-0.052022,0.011893,0.016401,0.006308,0.000170,0.006218,0.006130,-0.011372,-0.013668,-0.001557,0.011272,-0.010685,0.013971,0.019989,-0.003609,-0.019990,-0.025714,-0.020191,-0.086680,-0.000248,0.001047,0.003517,-0.004713,-0.039130,-0.001972,0.004885,0.007225,0.006474,-0.100246,0.007575,-0.002297,0.022440,0.020384,0.076434,-0.026420,-0.048723
2,0,"[-0.021778336, -0.007887523, -0.012167458, 0.0...",-0.021778,-0.007888,-0.012167,0.026645,-0.047003,0.041111,0.032634,0.007721,-0.004673,0.017945,0.025572,0.020721,-0.029443,0.007282,-0.012125,-0.005743,-0.021941,-0.026845,-0.008875,0.024187,-0.035593,0.016917,0.015894,0.000627,-0.027061,-0.008227,-0.012238,0.004773,0.005052,0.086486,-0.012999,0.013584,0.025601,-0.023260,-0.010884,-0.021703,0.011610,0.058846,...,0.013974,-0.027056,-0.014892,0.007018,0.004775,0.003739,-0.004609,0.015453,0.005407,0.002417,0.010540,-0.009061,-0.004683,-0.022841,-0.009695,-0.015350,-0.000986,0.000599,-0.002325,0.004891,0.000679,-0.023098,-0.053412,0.000287,-0.038986,0.012374,-0.007347,0.018700,0.030905,0.007683,0.001130,0.012536,-0.078928,0.052752,-0.023307,-0.003699,0.008551,0.036900,0.011529,0.001560
3,0,"[0.021224087, 0.003118416, -0.035745192, 0.044...",0.021224,0.003118,-0.035745,0.044600,-0.007005,-0.017203,0.036646,0.000290,0.003102,0.027911,-0.044804,-0.008287,0.004489,-0.016696,0.003103,0.000320,-0.033806,-0.013790,-0.012040,-0.013854,0.008394,0.004658,0.036740,0.023992,0.018495,-0.019312,0.013242,0.006967,-0.012553,0.111057,-0.001287,0.014520,-0.001761,-0.015324,-0.004771,0.005732,0.008985,0.033945,...,-0.024249,-0.016567,0.023114,0.011763,-0.032777,-0.000594,0.011036,0.010664,-0.001417,-0.000932,-0.023270,0.015315,-0.021258,-0.056209,0.026135,0.011350,0.022573,-0.003763,-0.043058,-0.008284,0.011969,0.016680,-0.055033,-0.025068,-0.000159,0.027143,-0.007409,-0.054329,-0.008116,0.003405,-0.017899,-0.001394,-0.123789,0.027707,0.001508,-0.001378,0.013516,0.063901,-0.025079,-0.014923
4,0,"[-0.04263019, 0.028402044, -0.02406443, 0.0112...",-0.042630,0.028402,-0.024064,0.011293,-0.001849,-0.013165,-0.012895,-0.024676,-0.038230,0.021767,-0.044259,-0.010850,0.051784,0.020780,-0.019907,-0.025127,0.008325,-0.010708,-0.043764,-0.035868,-0.055030,0.012830,-0.006759,0.015732,-0.050231,-0.014943,0.002229,0.049738,-0.021219,0.078146,0.012240,0.015609,-0.011149,-0.043478,-0.064961,0.013925,0.008646,-0.008385,...,0.040676,-0.015430,-0.018999,-0.002594,-0.047164,0.051087,0.005464,-0.019100,0.007361,-0.

Will be attempting xgboost here.

In [30]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
# split data into X and y
X = df3.iloc[:, 2:]
Y = df3['labelML']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=123)

In [33]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
12,-0.043814,0.003682,-0.054658,0.017873,-0.042610,0.033056,0.024984,0.021399,0.057089,0.048495,-0.027824,-0.011734,0.029167,-0.044032,0.074327,0.008636,0.073001,0.002442,-0.067387,-0.001765,0.009066,-0.018814,0.022079,0.045321,0.059291,0.018401,-0.038060,0.041943,0.014275,0.142638,-0.040212,-0.018686,-0.011764,-0.023034,-0.081277,0.017490,0.024777,0.054947,-0.046437,-0.035394,...,0.042133,-0.039009,0.106877,0.036133,-0.053906,0.035804,0.045760,-0.032677,0.052954,0.023920,0.038274,0.016015,-0.019670,-0.014643,0.014588,0.064390,-0.001419,0.062674,-0.028410,-0.022958,0.060895,-0.030631,-0.033990,0.009222,-0.020297,-0.044419,-0.078474,0.018232,0.031713,-0.005465,-0.032320,0.038996,-0.254619,0.111790,-0.008182,0.036008,0.046156,0.076570,-0.069858,-0.088116
0,0.000408,-0.006997,0.043368,0.059105,0.005750,-0.048923,-0.023493,-0.012002,0.008741,0.037747,-0.016657,0.021009,-0.000813,-0.005880,0.021754,0.012577,-0.026488,0.006688,-0.065610,0.031214,0.000651,0.027043,0.026573,-0.008683,0.011755,-0.033109,-0.012075,0.001186,-0.013979,0.112681,0.002063,-0.028648,-0.023800,0.026196,-0.012809,0.025506,0.010329,0.047436,-0.009022,0.035921,...,-0.010071,0.012518,0.041530,0.023292,-0.059176,0.013486,0.009417,0.022773,-0.007663,-0.022772,0.013969,0.001931,-0.034058,0.000787,0.023271,0.028486,0.013586,0.034805,-0.002466,0.012882,-0.038919,-0.005540,-0.085419,-0.050894,0.005842,0.044290,0.008440,-0.039907,-0.004749,0.006133,0.010734,0.009146,-0.105409,0.021976,-0.012046,0.027996,0.003418,0.054863,-0.031841,-0.042299
15,-0.020518,-0.009980,-0.013676,0.060925,-0.015802,-0.010306,0.020905,0.019526,0.016875,0.022405,-0.027124,-0.023696,-0.008668,-0.000685,0.011546,-0.018070,-0.001908,0.014303,-0.016892,0.002511,-0.005775,0.001321,0.002830,0.019206,0.013151,-0.018335,-0.020503,0.014213,-0.004832,0.097844,-0.010840,-0.015581,-0.009814,0.019432,-0.004442,0.007795,0.005518,0.034939,-0.036741,0.006313,...,-0.013506,-0.008860,0.023292,0.040557,-0.041923,0.018235,0.012392,0.010124,-0.011147,-0.002647,0.024562,0.011071,-0.042504,-0.017222,0.001353,0.004046,0.022996,0.020843,-0.037848,0.000267,0.034306,-0.022133,-0.057045,0.005524,0.004591,0.037417,-0.026321,-0.029632,-0.007204,-0.048110,-0.002584,0.001501,-0.119362,0.050021,0.021251,0.010614,0.000791,0.060911,-0.018204,-0.024803
16,-0.022313,0.010893,-0.007249,0.047847,-0.000565,0.032481,0.034530,-0.069139,0.020456,0.051890,-0.043708,-0.006780,-0.050142,-0.039754,-0.012689,0.018622,-0.011526,-0.004078,-0.006683,0.021269,-0.003059,0.078598,-0.008015,-0.033279,-0.013432,-0.022147,-0.047820,0.033264,-0.044413,0.085287,-0.002861,-0.046764,-0.004961,-0.038450,0.021536,0.034328,0.021598,-0.014047,0.027877,-0.012316,...,-0.029055,-0.015465,-0.036653,0.031436,-0.040623,0.027979,0.027139,0.000566,0.015445,-0.033922,-0.003457,0.001533,-0.007677,-0.019637,0.029579,-0.040860,-0.040107,-0.027294,-0.019180,0.042763,0.000914,-0.000690,-0.053477,-0.066109,-0.015556,-0.020976,0.054921,0.015876,-0.026000,-0.029375,0.023387,-0.004826,-0.037761,0.001078,0.010100,0.018460,-0.025779,0.037213,0.041407,0.025589
9,-0.006168,0.006165,0.017994,0.051674,-0.024459,-0.034630,0.014067,-0.011119,0.004933,0.032335,-0.040629,-0.003196,0.006606,0.005093,-0.030826,-0.012823,0.021207,-0.013278,-0.030477,0.047262,-0.005728,0.023553,0.002974,-0.019072,-0.038640,-0.020119,-0.027046,-0.000897,-0.013179,0.065766,-0.003564,-0.016765,-0.016408,-0.046633,-0.028873,-0.003590,0.005614,0.010530,-0.023557,-0.002585,...,0.010000,-0.010628,0.014558,0.013525,-0.047830,0.005807,0.028854,-0.015972,0.045571,-0.054742,-0.002308,-0.005432,-0.042873,-0.017697,-0.012324,-0.002472,0.017649,-0.007767,-0.021498,0.031840,-0.038362,-0.009441,-0.102836,-0.018020,-0.014665,0.004321,0.002779,-0.025225,-0

In [34]:
# Create arrays for the features and the target: X, y
X, y = df3['prediction'], df3['labelML']
#print(X.columns)

# Create the training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.285714


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

#tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
#>>> (tn, fp, fn, tp)

In [ ]:
type(X)

In [ ]:
y = dfready[['labelML']]

In [ ]:
type(y)

Keras NN model